<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto, Canada</font></h1>



## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1"> Transform Table on the Wikipedia page into the required Pandas Dataframe.</a>

2. <a href="#item2"> Use the Geocoder package or the csv file to create the dataframe: </a>

3. <a href="#item3"> Explore and Cluster Neighbourhoods </a>

</font>
</div>

In [1]:
!pip install bs4
from bs4 import BeautifulSoup
print("Done")

Done


In [2]:
!pip install lxml

In [3]:
import requests
import lxml.html as lh
import pandas as pd

## 1. **Transform Table on the Wikipedia page into the required Pandas Dataframe**

In [4]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url
#Get the contents of the web-page 
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')


In [5]:
# let’s parse the first row of the table as our header.
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    #There is a new-line character at the end of the last column. We need to remove it
    name = name.replace('\n','')
    print('%d:"%s"'%(i,name))
    col.append((name,[]))
    

1:"Postcode"
2:"Borough"
3:"Neighbourhood"


In [6]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [8]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        data = data.replace('\n','')
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1


In [9]:
#Just to be sure, let’s check the length of each column. Ideally, they should all be the same.
[len(C) for (title,C) in col]

[574, 574, 574]

#### _The wikipedia table converted to dataframe. **The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood**
--------------------------------------------------

In [10]:
# Let's create our Dataframe from the wikipedia data extracted
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### _Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned._
----------------------------------------------------------------------------------------------------------------

In [11]:

df = df[df['Borough'] != 'Not assigned'].reset_index(drop=True)
df.head()


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


#### _More than one neighborhood can exist in one postal code area. These two rows will be combined into one row with the neighborhoods separated with a comma._
--------------------------------------------------------------------------------------------------------------------------

In [12]:
df_grouped = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
df_grouped.head(20)



,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern,Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union,Highland ..."
2,M1E,Scarborough,"Guildwood,Morningside,West Hill,Guildwood,Morn..."
3,M1G,Scarborough,"Woburn,Woburn"
4,M1H,Scarborough,"Cedarbrae,Cedarbrae"
5,M1J,Scarborough,"Scarborough Village,Scarborough Village"
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park,East..."
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge,Clairlea,Golden ..."
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West,..."
9,M1N,Scarborough,"Birch Cliff,Cliffside West,Birch Cliff,Cliffsi..."


### _If a cell has a borough but a 'Not assigned' Neighborhood, then the Neighbourhood will be the same as the Borough._
-----------------------------------------------------------------------------------------------------------------------

In [13]:
# We need to use '.str.contains' to search for 'Not assigned,Not assigned...', as we combined Neighbourhoods with commas
df_grouped[df_grouped['Neighbourhood'].str.contains('Not assigned')]

,Postcode,Borough,Neighbourhood
93,M9A,Queen's Park,"Not assigned,Not assigned"


In [14]:
df_grouped.loc[df_grouped.Neighbourhood.str.contains('Not assigned'), 'Neighbourhood'] = df_grouped.Borough
# Lets check for Post M9A
df_grouped[df_grouped['Postcode'] == 'M9A']

,Postcode,Borough,Neighbourhood
93,M9A,Queen's Park,Queen's Park


### _Use the .shape method to print the number of rows of your dataframe._
--------------------------------------------------------------------------

In [15]:
df_grouped.shape

(103, 3)

In [16]:
print('The dataframe has {} Boroughs and {} Neighbourhoods.'.format(
        len(df_grouped['Borough'].unique()),
        df_grouped.shape[0]
    )
)


The dataframe has 11 Boroughs and 103 Neighbourhoods.


## _****************END OF PART-1*******************_
-----------------------------------------------------------

## 2. **Use the Geocoder package or the csv file to create the dataframe**



In [17]:

geo_data = pd.read_csv("http://cocl.us/Geospatial_data")
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
geo_data.shape

(103, 3)

In [19]:
# As the dataframe have the PostCode in the same order, we can just the columns directly
df_grouped['Latitude'] = geo_data['Latitude']


In [20]:
df_grouped.head()

,Postcode,Borough,Neighbourhood,Latitude
0,M1B,Scarborough,"Rouge,Malvern,Rouge,Malvern",43.806686
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union,Highland ...",43.784535
2,M1E,Scarborough,"Guildwood,Morningside,West Hill,Guildwood,Morn...",43.763573
3,M1G,Scarborough,"Woburn,Woburn",43.770992
4,M1H,Scarborough,"Cedarbrae,Cedarbrae",43.773136


In [21]:
df_grouped['Longitude'] = geo_data['Longitude']

### _The dataframe with Latitude and Longitude_
-----------------------------------------------



In [22]:
df_grouped.head(20)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern,Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union,Highland ...",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill,Guildwood,Morn...",43.763573,-79.188711
3,M1G,Scarborough,"Woburn,Woburn",43.770992,-79.216917
4,M1H,Scarborough,"Cedarbrae,Cedarbrae",43.773136,-79.239476
5,M1J,Scarborough,"Scarborough Village,Scarborough Village",43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park,East...",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge,Clairlea,Golden ...",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West,...",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West,Birch Cliff,Cliffsi...",43.692657,-79.264848


## _****************END OF PART-2*******************_
-----------------------------------------------------------

## 3. *Explore and Cluster Neighbourhoods*

In [23]:

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


#### _Explore and cluster the neighborhoods in Toronto_
-----------------------------------------------------------

In [24]:
# Slice  our dataframe and create a new dataframe for Toronto data 
toronto_data = df_grouped[df_grouped['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,"The Beaches,The Beaches",43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale,The Danforth West,...",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar,The Beaches West...",43.668999,-79.315572
3,M4M,East Toronto,"Studio District,Studio District",43.659526,-79.340923
4,M4N,Central Toronto,"Lawrence Park,Lawrence Park",43.728020,-79.388790


Let's get the geographical coordinates of Toronto.

In [25]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [26]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [27]:
CLIENT_ID = 'WPAN32M423EK3YBOUCSJLYFGCNUXVKL2AHRMRMZY5AJBMT1E' # your Foursquare ID
CLIENT_SECRET = 'VKOXRZZGL5YPST5MVQSMRM0NUADOE5V3VLZ5L3COWKZBZNCV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WPAN32M423EK3YBOUCSJLYFGCNUXVKL2AHRMRMZY5AJBMT1E
CLIENT_SECRET:VKOXRZZGL5YPST5MVQSMRM0NUADOE5V3VLZ5L3COWKZBZNCV


Get the FIRST neighbourhood's latitude and longitude values.

In [28]:
neighbourhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of The Beaches,The Beaches are 43.67635739999999, -79.2930312.


#### Now, let's get the top 100 venues of the first neighbourhood within a radius of 500 meters.

In [29]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius



# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=WPAN32M423EK3YBOUCSJLYFGCNUXVKL2AHRMRMZY5AJBMT1E&client_secret=VKOXRZZGL5YPST5MVQSMRM0NUADOE5V3VLZ5L3COWKZBZNCV&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

Send the GET request and examine the resutls

In [30]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e4aa9149da7ee001b9229ff'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

We know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [31]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [32]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Glen Stewart Park,Park,43.675278,-79.294647
4,Upper Beaches,Neighborhood,43.680563,-79.292869


Let's visualize the neighbourhoods in Toronto 

Start utilizing the Foursquare API to explore the neighbourhoods and segment them

#### Let's get the details to all the neighbourhoods in Toronto

In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Run the above function for each neighborhood and create a new dataframe called *Toronto_venues*.

In [34]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                 latitudes=toronto_data['Latitude'],
                                 longitudes=toronto_data['Longitude']
                                  )


The Beaches,The Beaches
The Danforth West,Riverdale,The Danforth West,Riverdale
The Beaches West,India Bazaar,The Beaches West,India Bazaar
Studio District,Studio District
Lawrence Park,Lawrence Park
Davisville North,Davisville North
North Toronto West,North Toronto West
Davisville,Davisville
Moore Park,Summerhill East,Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West,Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale,Rosedale
Cabbagetown,St. James Town,Cabbagetown,St. James Town
Church and Wellesley,Church and Wellesley
Harbourfront,Harbourfront
Ryerson,Garden District,Ryerson,Garden District
St. James Town,St. James Town
Berczy Park,Berczy Park
Central Bay Street,Central Bay Street
Adelaide,King,Richmond,Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station,Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre,Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Ho

#### Check the size of the resulting dataframe

In [35]:
print(toronto_venues.shape)
toronto_venues.head()

(1691, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"The Beaches,The Beaches",43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,"The Beaches,The Beaches",43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,"The Beaches,The Beaches",43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,"The Beaches,The Beaches",43.676357,-79.293031,Glen Stewart Park,43.675278,-79.294647,Park
4,"The Beaches,The Beaches",43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


#### Let's check how many venues were returned for each neighborhood

In [36]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide,King,Richmond,Adelaide,King,Richmond",100,100,100,100,100,100
"Berczy Park,Berczy Park",56,56,56,56,56,56
"Brockton,Exhibition Place,Parkdale Village,Brockton,Exhibition Place,Parkdale Village",22,22,22,22,22,22
"Business Reply Mail Processing Centre 969 Eastern,Business Reply Mail Processing Centre 969 Eastern",18,18,18,18,18,18
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara,CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",17,17,17,17,17,17
"Cabbagetown,St. James Town,Cabbagetown,St. James Town",41,41,41,41,41,41
"Central Bay Street,Central Bay Street",85,85,85,85,85,85
"Chinatown,Grange Park,Kensington Market,Chinatown,Grange Park,Kensington Market",85,85,85,85,85,85
"Christie,Christie",18,18,18,18,18,18


#### Let's find out how many unique categories can be curated from all the returned venues

In [37]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 228 uniques categories.


## Let's analyze each neighbourhood

In [38]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"The Beaches,The Beaches",0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,"The Beaches,The Beaches",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"The Beaches,The Beaches",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"The Beaches,The Beaches",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Beaches,The Beaches",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
toronto_onehot.shape

(1691, 229)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [40]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond,Adelaide,King,Richmond",0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,...,0.000000,0.000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.01,0.000000
1,"Berczy Park,Berczy Park",0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,"Brockton,Exhibition Place,Parkdale Village,Bro...",0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,Business Reply Mail Processing Centre 969 East...,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.055556
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.0000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,...,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,"Cabbagetown,St. James Town,Cabbagetown,St. Jam...",0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,"Central Bay Street,Central Bay Street",0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011765,0.000000,...,0.000000,0.000,0.00,0.011765,0.000000,0.000000,0.011765,0.000000,0.00,0.011765
7,"Chinatown,Grange Park,Kensington Market,Chinat...",0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000,0.00,0.035294,0.000000,0.047059,0.011765,0.000000,0.00,0.000000
8,"Christie,Christie",0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,"Church and Wellesley,Church and Wellesley",0.0125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012500,0.000000,...,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.012500,0.00,0.012500


#### Let's print each neighborhood along with the top 5 most common venues

In [41]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond,Adelaide,King,Richmond----
             venue  freq
0      Coffee Shop  0.06
1  Thai Restaurant  0.04
2             Café  0.04
3       Steakhouse  0.04
4       Restaurant  0.03


----Berczy Park,Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1     Cheese Shop  0.04
2  Farmers Market  0.04
3      Steakhouse  0.04
4          Bakery  0.04


----Brockton,Exhibition Place,Parkdale Village,Brockton,Exhibition Place,Parkdale Village----
                venue  freq
0                Café  0.14
1         Coffee Shop  0.09
2      Breakfast Spot  0.09
3                 Gym  0.05
4  Italian Restaurant  0.05


----Business Reply Mail Processing Centre 969 Eastern,Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.06
2                 Spa  0.06
3       Garden Center  0.06
4              Garden  0.06


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadi

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [43]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond,Adelaide,King,Richmond",Coffee Shop,Steakhouse,Thai Restaurant,Café,Bar,Cosmetics Shop,Sushi Restaurant,Burger Joint,Bakery,Restaurant
1,"Berczy Park,Berczy Park",Coffee Shop,Steakhouse,Seafood Restaurant,Cocktail Bar,Beer Bar,Bakery,Farmers Market,Cheese Shop,Café,Concert Hall
2,"Brockton,Exhibition Place,Parkdale Village,Bro...",Café,Coffee Shop,Breakfast Spot,Pet Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue,Bakery
3,Business Reply Mail Processing Centre 969 East...,Light Rail Station,Yoga Studio,Spa,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Pizza Place,Butcher
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Bar,Sculpture Garden


#### Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 3 clusters.

In [44]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [45]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,"The Beaches,The Beaches",43.676357,-79.293031,1,Park,Health Food Store,Pub,Neighborhood,Trail,Dog Run,Discount Store,Diner,Cupcake Shop,Doner Restaurant
1,M4K,East Toronto,"The Danforth West,Riverdale,The Danforth West,...",43.679557,-79.352188,1,Greek Restaurant,Italian Restaurant,Coffee Shop,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Frozen Yogurt Shop,Pub,Pizza Place,Liquor Store
2,M4L,East Toronto,"The Beaches West,India Bazaar,The Beaches West...",43.668999,-79.315572,1,Sandwich Place,Brewery,Ice Cream Shop,Fish & Chips Shop,Pub,Sushi Restaurant,Light Rail Station,Food & Drink Shop,Fast Food Restaurant,Italian Restaurant
3,M4M,East Toronto,"Studio District,Studio District",43.659526,-79.340923,1,Café,Coffee Shop,Gastropub,Bakery,Brewery,Italian Restaurant,American Restaurant,Neighborhood,Sandwich Place,Cheese Shop
4,M4N,Central Toronto,"Lawrence Park,Lawrence Park",43.728020,-79.388790,0,Park,Bus Line,Swim School,Event Space,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


Finally, let's visualize the resulting clusters

In [46]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine Clusters

#### Cluster 1

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,"Lawrence Park,Lawrence Park",0,Park,Bus Line,Swim School,Event Space,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
8,"Moore Park,Summerhill East,Moore Park,Summerhi...",0,Playground,Park,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
10,"Rosedale,Rosedale",0,Park,Playground,Trail,Cupcake Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store


#### Cluster 2

In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"The Beaches,The Beaches",1,Park,Health Food Store,Pub,Neighborhood,Trail,Dog Run,Discount Store,Diner,Cupcake Shop,Doner Restaurant
1,"The Danforth West,Riverdale,The Danforth West,...",1,Greek Restaurant,Italian Restaurant,Coffee Shop,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Frozen Yogurt Shop,Pub,Pizza Place,Liquor Store
2,"The Beaches West,India Bazaar,The Beaches West...",1,Sandwich Place,Brewery,Ice Cream Shop,Fish & Chips Shop,Pub,Sushi Restaurant,Light Rail Station,Food & Drink Shop,Fast Food Restaurant,Italian Restaurant
3,"Studio District,Studio District",1,Café,Coffee Shop,Gastropub,Bakery,Brewery,Italian Restaurant,American Restaurant,Neighborhood,Sandwich Place,Cheese Shop
5,"Davisville North,Davisville North",1,Park,Department Store,Gym,Breakfast Spot,Sandwich Place,Food & Drink Shop,Hotel,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
6,"North Toronto West,North Toronto West",1,Clothing Store,Coffee Shop,Bagel Shop,Gym / Fitness Center,Fast Food Restaurant,Diner,Dessert Shop,Mexican Restaurant,Park,Chinese Restaurant
7,"Davisville,Davisville",1,Sandwich Place,Dessert Shop,Pizza Place,Coffee Shop,Gym,Sushi Restaurant,Italian Restaurant,Café,Pharmacy,Diner
9,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",1,Coffee Shop,Pub,Sushi Restaurant,Fried Chicken Joint,Sports Bar,Pizza Place,Restaurant,American Restaurant,Liquor Store,Vietnamese Restaurant
11,"Cabbagetown,St. James Town,Cabbagetown,St. Jam...",1,Coffee Shop,Bakery,Café,Italian Restaurant,Restaurant,Pizza Place,Pub,Grocery Store,Japanese Restaurant,Sandwich Place
12,"Church and Wellesley,Church and Wellesley",1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Fast Food Restaurant,Pub,Gastropub,Men's Store,Mediterranean Restaurant


#### Cluster 3

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,"Roselawn,Roselawn",2,Garden,Yoga Studio,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


**Observations** - <em>k</em>-means clustering was used to partition the data into 5, 7 and 8 clusters, but each time, it was observed that most of the neighbourhoods belonged to one single cluster. So it can be concluded that the nighbourhoods in Toronto are quite similar to each other. Hence, for the final analysis, the data was grouped into just **3** clusters. 

Considering the common characteristics of each cluster, we could say that the neighbourhoods could be clustered as: 
- PARKS AND PLAYGROUNDS
- CAFE AND RESTAURANTS
- GARDEN 

The clusters 'PARKS AND PLAYGROUNDS' and 'GARDEN' seem very similar though. Both have Yoga studios, Dog runs, Eastern-Euorpean Restaurants and Dumpling Restaurants. 

## _****************END OF PART-3*******************_
-------------------------------------------------------